<a href="https://colab.research.google.com/github/ZhangYHe/MyColabDL_Repo/blob/main/work03_%E8%AE%AD%E7%BB%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##训练模型

在训练函数中首先初始化合成图像，计算原始风格图像的格拉姆矩阵列表，定义学习率下降调节器。在每一个epoch中计算合成图像的内容层和风格层的输出，再计算总损失值，反向传播，更新参数。训练结束后返回合成图像。


In [ ]:
def train(X, content_Y, style_Y, lr, num_epochs, lr_decay_epoch):
    # X是初始化的合成图像，style_Y_gram是原始风格图像的格拉姆矩阵列表
    X, style_Y_gram, trainer = get_inits(X, lr, style_Y)
    # 定义学习率下降调节器
    scheduler = torch.optim.lr_scheduler.StepLR(trainer, lr_decay_epoch, 0.8)
    for epoch in range(num_epochs):
        trainer.zero_grad()
        # Y_hat是用合成图像计算出的特征图
        content_Y_hat, style_Y_hat = extract_features(X, content_idx, style_idx)
        content_l, style_l, tv_l, l = compute_loss(X, content_Y, content_Y_hat, style_Y_gram, style_Y_hat)
        # 反向传播误差
        l.backward()
        # 更新一次合成图像的像素参数
        trainer.step()
        # 更新学习率超参数
        scheduler.step()
        # 每5个epoch记录一次loss信息
        if (epoch + 1) % 5 == 0:
            # 由于风格损失列表有5项，因此算个总损失输出
            print('迭代次数：{}    内容损失：{:.9f}    风格损失：{:.9f}    总变差损失：{:.9f}' .format(epoch+1, sum(content_l).item(), sum(style_l).item(), tv_l.item()))
    # 训练结束后返回合成图像
    return X


In [ ]:
output = train(X, content_features_Y, style_features_Y, lr = 0.3, num_epochs = 500, lr_decay_epoch = 50)
# 调用后处理函数处理最终的合成图像，将其转换为正常格式的可视化图像
output = postprocess(output)
# 显示图像
plt.imshow(output)
plt.show()


In [ ]:
# def train(X, contents_Y, styles_Y, device, lr, num_epochs, lr_decay_epoch):
#     X, styles_Y_gram, trainer = get_inits(X, device, lr, styles_Y)
#     scheduler = torch.optim.lr_scheduler.StepLR(trainer, lr_decay_epoch, 0.8)
#     animator = d2l.Animator(xlabel='epoch', ylabel='loss',
#                             xlim=[10, num_epochs],
#                             legend=['content', 'style', 'TV'],
#                             ncols=2, figsize=(7, 2.5))
#     for epoch in range(num_epochs):
#         trainer.zero_grad()
#         contents_Y_hat, styles_Y_hat = extract_features(
#             X, content_layers, style_layers)
#         contents_l, styles_l, tv_l, l = compute_loss(
#             X, contents_Y_hat, styles_Y_hat, contents_Y, styles_Y_gram)
#         l.backward()
#         trainer.step()
#         scheduler.step()
#         if (epoch + 1) % 10 == 0:
#             animator.axes[1].imshow(postprocess(X))
#             animator.add(epoch + 1, [float(sum(contents_l)),
#                                      float(sum(styles_l)), float(tv_l)])
#     return X

In [ ]:
# device, image_shape = d2l.try_gpu(), (300, 450)
# net = net.to(device)
# content_X, contents_Y = get_contents(image_shape, device)
# _, styles_Y = get_styles(image_shape, device)
# output = train(content_X, contents_Y, styles_Y, device, 0.3, 500, 50)